In [24]:
import pandas as pd

matches = pd.read_csv("matches.csv", index_col=0)
matches.head()

matches.shape

# 2 seasons * 20 squads * 38 matches

2 * 20 * 38


# Missing Liverpool 2021-2022
matches["team"].value_counts()


matches[matches["team"] == "Liverpool"].sort_values("date")



matches["round"].value_counts()


matches.dtypes



del matches["comp"]
del matches["notes"]
matches["date"] = pd.to_datetime(matches["date"])
matches["target"] = (matches["result"] == "W").astype("int")
matches



matches["venue_code"] = matches["venue"].astype("category").cat.codes
matches["opp_code"] = matches["opponent"].astype("category").cat.codes
matches["hour"] = matches["time"].str.replace(":.+", "", regex=True).astype("int")
matches["day_code"] = matches["date"].dt.dayofweek
matches



from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)
train = matches[matches["date"] < '2022-01-01']
test = matches[matches["date"] > '2022-01-01']
predictors = ["venue_code", "opp_code", "hour", "day_code"]
rf.fit(train[predictors], train["target"])
RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)
preds = rf.predict(test[predictors])

from sklearn.metrics import accuracy_score
error = accuracy_score(test["target"], preds)
error



combined = pd.DataFrame(dict(actual=test["target"], predicted=preds))
pd.crosstab(index=combined["actual"], columns=combined["predicted"])


from sklearn.metrics import precision_score

precision_score(test["target"], preds)



grouped_matches = matches.groupby("team")
group = grouped_matches.get_group("Manchester City").sort_values("date")
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("date")
    rolling_stats = group[cols].rolling(3, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group




cols = ["gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt"]
new_cols = [f"{c}_rolling" for c in cols]

rolling_averages(group, cols, new_cols)


matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))
matches_rolling


matches_rolling = matches_rolling.droplevel('team')

matches_rolling

C:\Users\hamza\AppData\Local\Temp\ipykernel_59844\4067689100.py:88: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))


date   time         round  day venue result  \
team                                                                           
Arsenal                 29 2020-02-19  15:00  Matchweek 26  Sat  Home      W   
                        30 2020-02-24  19:45  Matchweek 20  Thu  Home      W   
                        31 2020-03-06  14:00  Matchweek 28  Sun  Away      W   
                        32 2020-03-13  16:30  Matchweek 29  Sun  Home      W   
                        33 2020-03-16  20:15  Matchweek 27  Wed  Home      L   
...                               ...    ...           ...  ...   ...    ...   
Wolverhampton Wanderers 15 2023-12-01  19:30  Matchweek 14  Wed  Home      D   
                        16 2023-12-04  15:00  Matchweek 15  Sat  Home      L   
                        17 2023-12-11  12:30  Matchweek 16  Sat  Away      L   
                        18 2023-12-15  19:30  Matchweek 17  Wed  Away      W   
                        19 2023-12-19  14:00  Matchweek 18  Sun  Home      D   

                             gf   ga         opponent   xg  ...  hour  \
team                                                        ...         
Arsenal                 29  2.0  1.0        Brentford  2.1  ...    15   
                        30  2.0  1.0           Wolves  2.7  ...    19   
                        31  3.0  2.0          Watford  1.4  ...    14   
                        32  2.0  0.0   Leicester City  2.3  ...    16   
                        33  0.0  2.0        Liverpool  0.5  ...    20   
...                         ...  ...              ...  ...  ...   ...   
Wolverhampton Wanderers 15  0.0  0.0          Burnley  0.7  ...    19   
                        16  0.0  1.0        Liverpool  0.2  ...    15   
                        17  0.0  1.0  Manchester City  0.2  ...    12   
                        18  1.0  0.0         Brighton  1.3  ...    19   
                        19  0.0  0.0          Chelsea  0.3  ...    14   

                            day_code  gf_rolling ga_rolling sh_rolling  \
team                                                                     
Arsenal                 29         2    0.666667   0.666667  13.000000   
                        30         0    1.000000   0.333333  18.666667   
                        31         4    1.666667   0.666667  20.666667   
                        32         4    2.333333   1.333333  22.000000   
                        33         0    2.333333   1.000000  20.666667   
...                              ...         ...        ...        ...   
Wolverhampton Wanderers 15         4    0.333333   0.666667   8.333333   
                        16         0    0.333333   0.000000  12.000000   
                        17         0    0.000000   0.333333   8.000000   
                        18         4    0.000000   0.666667   7.000000   
                        19         1    0.333333   0.666667   4.333333   

                           sot_rolling dist_rolling  fk_rolling  pk_rolling  \
team                                                                          
Arsenal                 29    3.000000    17.066667    0.666667    0.000000   
                        30    4.666667    17.133333    0.666667    0.000000   
                        31    4.666667    16.066667    0.333333    0.000000   
                        32    5.333333    16.000000    0.333333    0.000000   
                        33    5.000000    16.066667    0.333333    0.333333   
...                                ...          ...         ...         ...   
Wolverhampton Wanderers 15    3.000000    17.800000    0.333333    0.000000   
                        16    3.333333    18.600000    0.000000    0.000000   
                        17    2.000000    20.500000    0.000000    0.000000   
                        18    1.666667    18.033333    0.000000    0.000000   
                        19    2.333333    16.800000    0.333333    0.000000   

                            pkatt_rolling  
team   